In [ ]:
# this piece of code converts the raw csv file to processed input
import os
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd

from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

porter = PorterStemmer()

#读取news_data.csv，保存到新建的news_data.txt中
data = pd.read_csv('earbud_reviews.csv', encoding='utf-8')
#stop_words = set(stopwords.words('english'))

a = 0
LL = {} # dict to store num of appearance
punctuation_string = string.punctuation # store the punctuation
'''
for line in data.values:
    stri = str(line[13]) # this is the review body
    stri = stri.replace("  ", " ")
    L = nltk.word_tokenize(stri) # tokenlize 
    #L = stri.split(" ") # split the review body by space
    for word in L:
        if word.isalnum():
            stemmed = porter.stem(word.lower()) # stem the word
            if stemmed in LL:
                LL[stemmed] += 1
            else:
                LL[stemmed] = 0
            #LL.append(porter.stem(word)) # append each word of each review body to LL
    a += 1
    print(a)
    
#print(LL)
with open('temp.txt','a+', encoding='utf-8') as f:
    for key in LL.keys():
        f.write(str(key) + " " + str(LL[key]) + "\n")
    f.close()

'''
with open('temp.txt') as f:
    dataa = f.readlines()
    for line in dataa:
        line = line.split(" ")
        LL[line[0]] = int(line[1])
    f.close()


with open('earbud.txt','a+', encoding='utf-8') as f: # store the data in this file
    with open('new_vote.txt','a+', encoding = 'utf-8') as ff:
        with open('o.txt', 'a+', encoding = 'utf-8') as fff:
            temp = 0 
            for line in data.values:
                #if temp < 100:

                stri = str(line[13])
                '''
                for i in punctuation_string:
                    stri = stri.replace(i, '') # replace punctuation with nothing
                stri = stri.replace("  ", " ")
                stri = stri.lower() # convert to lower case
                L = stri.split(" ") # split by space
                '''
                L = nltk.word_tokenize(stri)
                LLL = [word.lower() for word in L if word.isalnum()]
                if len(LLL) > 1:
                    filtered_words = []
                    for word in LLL:
                        stemmed = porter.stem(word)
                        if (stemmed not in stopwords.words('english')) and (stemmed in LL) and (LL[stemmed] >= 10):
                            filtered_words.append(stemmed)
                    #filtered_words = [porter.stem(word) for word in L if (porter.stem(word) not in stopwords.words('english')) and (LL.count(word)) >= 10] # keep only if appear more than 10 times and not stop words
                    new = " ".join(filtered_words)
                    #f.write(str(line[1]) + " " + str(line[2]) + " " + new + " " + stri + " " + str(line[8]) + " " + str(int(line[9]) - int(line[8])) + '\n')
                    #if new.isalnum():

                    t = str(line[1]) + "_" + str(line[2]) + " " + str(line[7]) + " " + new
                    processed_content = t.split(" ", 2)[2]
                    if processed_content != "" and processed_content != " " and processed_content != "\t":
                        f.write(t + '\n')
                        fff.write(str(line[13])+"\n")
                        ff.write((str(line[8]) + " " + str(int(line[9]) - int(line[8]))+ "\n"))
                    
                        # line[1] is customer_id 
                        # line[2] is review_id
                        temp+=1
                        print(temp)
            fff.close()
        ff.close()
    f.close()
            
        #temp += 1



In [ ]:
# write all the data into csv file

#customer id, review id, actual rating, predicted rating, processed content, original content, up vote, down vote, prob 1-10, entropy, review length

# earbud_reviews.txt: customer_id review_id
import csv  
import nltk
from scipy.stats import entropy
file = open("AIR_document_topic") # output file

LL = []
header = ['Cutomer ID', 'Review ID', 'Actual Rating', 'Predicted Rating', 'Processed Content', 'Original Content', 'Up Vote', 'Down Vote',
    'prob_a1', 'prob_a2', 'prob_a3', 'prob_a4', 'prob_a5', 'prob_a6', 'prob_a7', 'prob_a8', 'prob_a9', 'prob_a10', 'Entropy', 'Review Length']

LL.append(header)
topic_score = file.readlines() # extract data


with open("filtered_tokenlized.txt") as file2:
    info = lines = [line.rstrip('\n') for line in file2]
    # info is all the lines

file3 = open("AIR_score") # predicted score by the algo

f3 = file3.readlines()

file4 = open("o.txt")
f4 = file4.readlines()

file5 = open("new_vote.txt")
f5 = file5.readlines()

for i in range(630526): # number of reviews after filter
    L = []
    temp = topic_score[i].split(" ")[0].split("\t") # this is each score
    t = info[i] # info for ith line
    tempp = [int(x) for x in temp] # convert str to int
    summation = sum(tempp) # sum of all the numbers 

    processed_content = t.split(" ", 2)[2]


    
    score = f3[i][:-1].split("\t")
    scoree = [float(x) for x in score][1:]

    predicted_score = sum(scoree)/10

    cus_review_id = t.split(" ", 1)[0].split("_")

    original_content = f4[i].rstrip("\n")

    up_down = f5[i].rstrip("\n").split(" ")
    upvotes = int(up_down[0])
    downvotes = int(up_down[1])

    ent = entropy(list(tempp), base=2)

    words = nltk.word_tokenize(original_content)
    words=[word for word in words if word.isalnum()]

    

    L.append(cus_review_id[0])
    L.append(cus_review_id[1])
    L.append(score[0])
    L.append(predicted_score)
    L.append(processed_content)
    # original content
    L.append(original_content)
    L.append(upvotes)
    L.append(downvotes)

    for num in tempp:
        L.append("%.2f" % (num/summation*100) + "%" + "  ")

    L.append(ent)
    L.append(len(words)) # review length

    LL.append(L)
    print(i)
print(len(LL))


with open('countries.csv', 'a+', encoding='UTF8') as f:
    writer = csv.writer(f)

    writer.writerows(LL)


f.close()


In [17]:
import pandas as pd
df = pd.read_csv('new.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630526 entries, 0 to 630525
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Cutomer ID         630526 non-null  int64  
 1   Review ID          630526 non-null  object 
 2   Actual Rating      630526 non-null  float64
 3   Predicted Rating   630526 non-null  float64
 4   Processed Content  630526 non-null  object 
 5   Original Content   630526 non-null  object 
 6   Up Vote            630526 non-null  int64  
 7   Down Vote          630526 non-null  int64  
 8   prob_a1            630526 non-null  object 
 9   prob_a2            630526 non-null  object 
 10  prob_a3            630526 non-null  object 
 11  prob_a4            630526 non-null  object 
 12  prob_a5            630526 non-null  object 
 13  prob_a6            630526 non-null  object 
 14  prob_a7            630526 non-null  object 
 15  prob_a8            630526 non-null  object 
 16  pr

In [ ]:
# below are some pieces of code to test 

In [ ]:
# code to upload file

# scp Desktop/earbud_reviews/filtered_tokenlized.txt sunyt@theorymind.d2.comp.nus.edu.sg:/home/sunyt/AIR/AIR/data2

In [ ]:
test = '24	54	6	3	63	9	54'
temp = test.split(" ")[0].split("\t")
print(temp)

In [ ]:
with open("earbud_reviews.txt") as file2:
    info = lines = [line.rstrip('\n') for line in file2]
    infoo = info[1].split(" ", 2)[2]
    print(infoo)
    file2.close()


In [ ]:
test = '2.0	2.271	5.142	2.343	2.463	1.605	4.714	0.657'
temp = test[:-1].split("\t")
print(temp)


In [ ]:
import pandas as pd
import string


data = pd.read_csv('earbud_reviews.csv', encoding='utf-8')
punctuation_string = string.punctuation

with open('origin.txt','a+', encoding='utf-8') as f: 
    temp = 0 
    for line in data.values:
            stri = str(line[13])
            for i in punctuation_string:
                stri = stri.replace(i, '') # replace punctuation with nothing
            stri = stri.replace("  ", " ")
            stri = stri.lower() # convert to lower case
            L = stri.split(" ") # split by space
            if len(L) > 1:
                f.write(stri + "\n")
                temp+=1
                print(temp)
f.close()


In [ ]:
with open("earbud_reviews") as file2:
    file = open("AIR_document_topic") # output file

    topic_score = file.readlines() # extract data
    temp = 0
    info = [line.rstrip('\n') for line in file2]
    
    for i in range(630929):
        t = info[i]
        processed_content = t.split(" ", 2)[2]
        processed_content = processed_content.split(" ")
        tempp = topic_score[i].split(" ")[0].split("\t") # this is each score
        temppp = [int(x) for x in tempp] # convert str to int
        summation = sum(temppp) # sum of all the numbers 
        if len(processed_content) != summation:
            print(t)
            print(topic_score[i])
            print(summation)
            print(len(processed_content))
            break
    file2.close()
        
    
    

In [ ]:
import pandas as pd
data = pd.read_csv('earbud_reviews.csv', encoding='utf-8').values
#print(data)
with open("earbud.txt") as file2:
    with open("filtered_votes.txt", 'a+', encoding='utf-8') as file3:
        with open('filtered_tokenlized.txt','a+', encoding='utf-8') as f: 
            temp = 0
            info = [line.rstrip('\n') for line in file2]
            for i in range(630651):
                t = info[i]
                processed_content = t.split(" ", 2)[2]
                if processed_content != "" and processed_content != " " and processed_content != "\t":
                    f.write(info[i] + "\n")
                    file3.write(str(data[i][8]) + " " + str(int(data[i][9]) - int(data[i][8]))+ "\n")
                    temp+=1
                print(temp)
            f.close()
        file3.close()
    file2.close()
        

In [ ]:
import nltk

stri = "Simple hanger; clamp is strong, with a pad; hanger swivels to accommodate many possible attachment locations"

L = nltk.word_tokenize(stri)
print(L)


In [ ]:
data = pd.read_csv('earbud_reviews.csv', encoding='utf-8')
#stop_words = set(stopwords.words('english'))

a = 0
LL = {} # dict to store num of appearance
punctuation_string = string.punctuation # store the punctuation
for line in data.values:
    stri = str(line[13]) # this is the review body
    stri = stri.replace("  ", " ")
    L = nltk.word_tokenize(stri) # tokenlize 
    LL = [word for word in L if word not in punctuation_string]
    print(LL)

In [ ]:
file4 = open("filtered_origin.txt")
f4 = file4.readlines()

L = []
for i in range(100):
    L.append(f4[i])

print(L)

In [ ]:
# this piece of code converts the raw csv file to processed input
import os
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd

from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

porter = PorterStemmer()

#读取news_data.csv，保存到新建的news_data.txt中
data = pd.read_csv('earbud_reviews.csv', encoding='utf-8')
#stop_words = set(stopwords.words('english'))

a = 0
punctuation_string = string.punctuation # store the punctuation

    
file2 = open("filtered_tokenlized.txt", "r")
dataa = file2.readlines()


with open('earbud.txt','a+', encoding='utf-8') as f: # store the data in this file
    with open("new_vote.txt",'a+', encoding='utf-8') as ff:
        temp = 0 
        #if temp < 100:
        for linee in dataa:
            cus_review_id = linee.split(" ", 1)[0].split("_")
            cus_id = cus_review_id[0]
            review_id = cus_review_id[1]

            for line in data.values:

                if line[0] == cus_id and line[1] == review_id:
                    f.write(line[13])
                    ff.write((str(data[i][8]) + " " + str(int(data[i][9]) - int(data[i][8]))+ "\n"))


                    temp+=1
                    print(temp)
                    continue
    

file2.close()  
        #temp += 1



In [ ]:
line = "2781942_R1DGA6UQIVLKZ7 5 alll good\n"
print(line.split(" ", 1)[0].split("_"))